In [2]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.wait import WebDriverWait
from pathlib import Path


from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException 
from selenium.webdriver.support import expected_conditions as EC
import requests


# Step 1: Create a session and load the page
epm = 'https://dunksandthrees.com/epm?season=2023'

#url_list = [url1,url2,url3,url4,url5]

def check_exists_by_xpath(driver, xpath):
    try:
        driver.find_element(By.XPATH, xpath)
    except NoSuchElementException:
        return False
    return True
#url_list = [cs,pullup]
def get_tables(url_list):
    data = []
    driver = webdriver.Chrome()
    for url in url_list:
        print(url)
        
        driver.get(url)
        w_path = '//*[@id="player-stats"]/div[2]/table/tbody/tr[480]'
        xpath = '//*[@id="player-stats"]/div[2]/table'
        #button_path = '/html/body/div/main/div/div[3]/div[2]/table/thead/tr[2]/th[8]'
        '''
        try:
            elem = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.XPATH,xpath)))
            
        finally:
            driver.quit()
        
        '''
        #'''
        wait = WebDriverWait(driver, 10)
        result = wait.until(EC.presence_of_element_located((By.XPATH,w_path)))
        elem = driver.find_element(By.XPATH,xpath)
        
        # Wait for the page to fully load
        #print(elem.text)
        #elem = driver.find_element(By.XPATH,xpath)
        #driver.implicitly_wait(40)
        #driver.execute_script("window.scrollTo(0,document.body.scrollHeight)",elem)
        driver.implicitly_wait(40)
       
        '''if check_exists_by_xpath(driver, "//a[contains(text(),'>')]/preceding-sibling::a[1]"):
            number_of_pages = int(driver.find_element(By.XPATH, "//a[contains(text(),'>')]/preceding-sibling::a[1]").text)
            print(number_of_pages)'''
        # Step 2: Parse HTML code and grab tables with Beautiful Soup
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        
        #return elem
        #rows = soup.find_all('tr')
        #print(rows)
        #return rows
        tables = soup.find_all('table')
        #vals = soup.find_all('td',class_ = 'player_name')
        #print(len(vals))
        #print(vals)
        # Step 3: Read tables with Pandas read_html()
        df = pd.read_html(str(tables))[0]
        #print(dfs)
        data.append(df)
    driver.close()
    return data
elem = get_tables([epm])

https://dunksandthrees.com/epm?season=2023


TimeoutException: Message: 
Stacktrace:
Backtrace:
	(No symbol) [0x006C37D3]
	(No symbol) [0x00658B81]
	(No symbol) [0x0055B36D]
	(No symbol) [0x0058D382]
	(No symbol) [0x0058D4BB]
	(No symbol) [0x005C3302]
	(No symbol) [0x005AB464]
	(No symbol) [0x005C1215]
	(No symbol) [0x005AB216]
	(No symbol) [0x00580D97]
	(No symbol) [0x0058253D]
	GetHandleVerifier [0x0093ABF2+2510930]
	GetHandleVerifier [0x00968EC1+2700065]
	GetHandleVerifier [0x0096C86C+2714828]
	GetHandleVerifier [0x00773480+645344]
	(No symbol) [0x00660FD2]
	(No symbol) [0x00666C68]
	(No symbol) [0x00666D4B]
	(No symbol) [0x00670D6B]
	BaseThreadInitThunk [0x76867D69+25]
	RtlInitializeExceptionChain [0x7796BB9B+107]
	RtlClearBits [0x7796BB1F+191]


In [ ]:
df = elem[0]
df.columns = df.columns.droplevel()
df = df.dropna(subset = 'PLAYER')
df = df[df.PLAYER != 'PLAYER']
df.to_csv('epm_temp.csv',index = False)
df = pd.read_csv('epm_temp.csv')

In [5]:

new_columns = []
columns = df.columns
for col in columns:
   # print(col)
    #print(df[col].dtype)
    if df[col].dtype == 'object':
       
        if '%' in df.loc[1][col]:
            df[[col,col+'_per']] = df[col].str.split("%",expand=True)
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col+'_per'] = pd.to_numeric(df[col+'_per'], errors='coerce')
#df[['PLAYER','POS']] = df['PLAYER'].str.split('·',expand = True)
#df['PLAYER'] = df['PLAYER'].str.replace(' ', '')
df['pos'] = df['PLAYER'].str.split("·",expand = True)[1]

df['PLAYER'] = df['PLAYER'].str.split("·",expand = True)[0]
df['team'] = df['PLAYER'].str[-4:-1]
df['PLAYER'] =  df['PLAYER'].str[:-4]
df = df.round(2)
df.columns = [x.lower() for x in df.columns]

In [6]:
df.to_csv('epm/epm.csv',index = False)


In [7]:

def scrape_LEBRON():
    url = 'https://www.bball-index.com/lebron-database/'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    table = soup.find(id = 'table_1')
    df = pd.read_html(str(table),displayed_only=False)[0]
    df = df.dropna(subset = 'Player')
    year = df['Season'].str[0:4].astype(int) +1
    df['year'] = year
    df['Player'] = df['Player'].str.lower()
    df = df.rename(columns = {'Team':'team'})
    return df
df = scrape_LEBRON()


df = df[df['Season'] =='2022-23']
df.to_csv('lebron/lebron.csv',index = False)
df.to_csv('2023/lebron/lebron.csv',index = False)

In [10]:
'''
for i in range(2013,2022):
        


 
    path = str(i+1)+'/lebron/'
    output_dir = Path(path)
    output_dir.mkdir(parents=True, exist_ok=True)
    frame = df[df['year'] ==i+1]
    frame.to_csv(str(i+1)+'/lebron/lebron.csv',index = False)'''

In [7]:
df

,bref_id,Player,Season,team,Offensive Archetype,Minutes,Games,Pos,Age,LEBRON,O-LEBRON,D-LEBRON,WAR,NBA ID,boxLEBRON,boxOLEBRON,boxDLEBRON,year
6517,jokicni01,nikola jokic,2022-23,DEN,Post Scorer,1266,38,C,27,7.01,5.49,1.52,7.30,203999.0,5.94,4.89,1.05,2023
6518,doncilu01,luka doncic,2022-23,DAL,Shot Creator,1497,40,PG,23,5.85,6.16,-0.31,7.70,1629029.0,6.49,6.31,0.17,2023
6519,tatumja01,jayson tatum,2022-23,BOS,Shot Creator,1542,42,PF,24,4.94,4.62,0.32,7.10,1628369.0,5.08,4.57,0.51,2023
6520,jacksja02,jaren jackson jr.,2022-23,MEM,Stretch Big,679,26,C,23,4.94,1.16,3.77,3.10,1628991.0,4.00,0.67,3.33,2023
6521,davisan02,anthony davis,2022-23,LAL,Post Scorer,836,25,C,29,4.92,2.86,2.06,3.90,203076.0,4.74,2.97,1.77,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7004,nnajize01,zeke nnaji,2022-23,DEN,Roll + Cut Big,369,32,PF,22,-3.30,-1.94,-1.36,-0.20,1630192.0,-2.34,-1.47,-0.87,2023
7005,houseda01,danuel house jr.,2022-23,PHI,Stationary Shooter,524,32,SF,29,-3.36,-2.30,-1.05,-0.30,1627863.0,-2.45,-2.01,-0.44,2023
7006,sharpsh01,shaedon sharpe,2022-23,POR,Athletic Finisher,808,41,SG,19,-3.43,-1.89,-1.54,-0.50,1631101.0,-2.91,-1.78,-1.13,2023
7007,branhma01,malaki branham,2022-23,SAS,Secondary Ball Handler,563,29,SF,19,-3.53,-1.52,-2.01,-0.40,1631103.0,-3.50,-2.01,-1.48,2023
